In [36]:
from pynq.overlays.base import BaseOverlay
import numpy as np
from time import sleep
from scipy import signal
from IPython.display import Audio
import pandas as pd
base = BaseOverlay("base.bit")
pAudio = base.audio

In [37]:
Delay1 = 0.3
color = 4
rgbled_position = [4,5]

In [38]:
def light_LEDs():
    for led in rgbled_position:
        base.rgbleds[led].write(color)
        base.rgbleds[led].write(color)
def LEDs_off():
    for led in rgbled_position:
        base.rgbleds[led].off()

In [45]:
def record():
    pAudio.record(2)
    pAudio.save("Recording_1.pdm")
    pAudio.load("Recording_1.pdm")

In [40]:
def convert():
    af_uint8 = np.unpackbits(pAudio.buffer.astype(np.int16)
                         .byteswap(True).view(np.uint8))

    af_dec = signal.decimate(af_uint8,8,zero_phase=True)
    af_dec = signal.decimate(af_dec,6,zero_phase=True)
    af_dec = signal.decimate(af_dec,2,zero_phase=True)
    af_dec = (af_dec[10:-10]-af_dec[10:-10].mean())

    del af_uint8
    return af_dec

In [46]:
light_LEDs()
record()
LEDs_off()
af_dec = convert()
df = pd.DataFrame(af_dec)
df.to_csv("recording.csv")

/usr/lib/python3/dist-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]
/usr/lib/python3/dist-packages/scipy/signal/signaltools.py:3424: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return y[sl]


In [47]:
Audio(af_dec, rate=32000)

In [35]:
print(af_dec.shape)

(95980,)
